In [7]:
import pandas as pd
import os
import pyLDAvis
import pyLDAvis.gensim
import jieba.posseg as pseg
import nltk
from gensim import corpora, models, similarities

In [8]:
base_dir = './数据集'
train_dir = os.path.join(base_dir, '新闻分类.train.txt')
test_dir = os.path.join(base_dir, '新闻分类.test.txt')
val_dir = os.path.join(base_dir, '新闻分类.val.txt')
vocab_dir = os.path.join(base_dir, '新闻分类.vocab.txt')
df = pd.read_csv(train_dir, sep='\t', header=None, names=['Category', 'Text'])
df

,Category,Text
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,体育,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,体育,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,体育,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,体育,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...
...,...,...
49995,财经,打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...
49996,财经,昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...
49997,财经,沪基指全周大涨8.25% 创两个月最大周涨幅全景网2月6日讯 受到A股市场牛年高歌猛进影响，...
49998,财经,沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 假设已有一个包含多个类别和文本的数据集 df

# 添加新文章到数据集
new_article = pd.DataFrame({'Category': ['财经'], 'Text': ["东吴进取策略基金获准发行东吴基金旗下第五只产品——东吴进取策略混合型基金获得中国证监会批文，将于近期正式发行。东吴进取策略基金为灵活配置混合型基金，其中股票投资比例为基金资产的30%-80%，债券投资比例为0-70%，现金或到期日在一年期以内的政府债券不低于5%。基金经理由东吴基金投资总监、东吴双动力基金经理王炯兼任。东吴进取策略基金以成长型股票为主要投资标的，但与其他基金不同的是，该基金从企业内涵式增长和外延式增长两个层面，将成长型股票分成长期快速成长型、周期成长型和转型成长型三类。王炯表示，这三类不同成长类型的股票，在不同经济周期下会有不同的风险收益状况。对此，东吴进取策略基金将采取不同的投资策略，其中，对长期快速成长型股票，可以采取中长期买入持有操作策略；对周期成长型股票，采取周期持有操作策略；对转型成长型股票，则可以采取中短期买入持有操作策略。(李良)"]})
df = pd.concat([df, new_article], ignore_index=True)

# 文本预处理
df = df[df['Category'] == '财经'].copy()
# df['Text'] = df['Text'].apply(lambda x: x.lower())

# 特征提取
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text'])

# 计算余弦相似性
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# 找到新文章在数据集中的索引
new_article_index = df.shape[0] - 1

# 推荐函数
def recommend(index):
    # 获取相似度分数
    sim_scores = list(enumerate(cosine_similarities[index]))
    # 根据相似度分数排序
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 获取前3个推荐
    top_similar_indices = [i[0] for i in sim_scores[1:4]]
    return df.iloc[top_similar_indices]

# 以新文章为例进行推荐
recommendations = recommend(new_article_index)
print(recommendations)


     Category                                               Text
3093       财经  正在招募的债券型基金荆楚网消息 (楚天都市报) 基金名称 认购费率 发行结束日 债券投资比例...
4692       财经  中银行业优选基金获准发行今日，中银基金2009年第一只基金产品——中银行业优选灵活配置混合型...
2917       财经  诺德主题灵活配置基金即将发售⊙本报记者 徐婧婧 诺德基金今日公告，旗下诺德主题灵活配置混合型...


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

base_dir = './数据集'
train_dir = os.path.join(base_dir, '新闻分类.train.txt')
test_dir = os.path.join(base_dir, '新闻分类.test.txt')
val_dir = os.path.join(base_dir, '新闻分类.val.txt')
vocab_dir = os.path.join(base_dir, '新闻分类.vocab.txt')
df = pd.read_csv(train_dir, sep='\t', header=None, names=['Category', 'Text'])
df
# 添加新文章到数据集
new_article = pd.DataFrame({'Category': ['财经'], 'Text': ["东吴进取策略基金获准发行东吴基金旗下第五只产品——东吴进取策略混合型基金获得中国证监会批文，将于近期正式发行。东吴进取策略基金为灵活配置混合型基金，其中股票投资比例为基金资产的30%-80%，债券投资比例为0-70%，现金或到期日在一年期以内的政府债券不低于5%。基金经理由东吴基金投资总监、东吴双动力基金经理王炯兼任。东吴进取策略基金以成长型股票为主要投资标的，但与其他基金不同的是，该基金从企业内涵式增长和外延式增长两个层面，将成长型股票分成长期快速成长型、周期成长型和转型成长型三类。王炯表示，这三类不同成长类型的股票，在不同经济周期下会有不同的风险收益状况。对此，东吴进取策略基金将采取不同的投资策略，其中，对长期快速成长型股票，可以采取中长期买入持有操作策略；对周期成长型股票，采取周期持有操作策略；对转型成长型股票，则可以采取中短期买入持有操作策略。(李良)"]})
df = pd.concat([df, new_article], ignore_index=True)

# 文本预处理
df = df[df['Category'] == '财经'].copy()

# 特征提取 - 使用词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Text'])

# 使用LDA进行主题建模
num_topics = 5  # 你可以根据实际需求调整主题数量
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# 将新文章映射到主题分布
new_article_vector = vectorizer.transform([new_article['Text'].iloc[0]])
new_article_topic_distribution = lda.transform(new_article_vector)

# 找到与新文章主题分布相似的文章
similar_articles_indices = (pd.DataFrame(lda.transform(X))
                           .apply(lambda x: sum((x - new_article_topic_distribution.flatten()) ** 2), axis=1)
                           .nsmallest(4).index)

# 输出推荐的文章
recommendations = df.iloc[similar_articles_indices]
print(recommendations[1:4])


      Category                                               Text
48758       财经  华夏平稳 华夏蓝筹两基金继续暂停申购业务全景网11月21日讯 华夏基金今日发布公告称，公司旗...
49196       财经  中银增利开放申购业务中银基金公告，公司首只债券型基金——中银增利基金即日起开放申购业务。中银...
46092       财经  宁波现第三套人民币1960版贰元券假币□据新华社报道记者15日从浙江省经侦部门获悉，2008...
